<a href="https://colab.research.google.com/github/MaryamMortazaviSefat/new/blob/main/q2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#load data
!wget -q https://raw.githubusercontent.com/MaryamMortazaviSefat/practice2_Prediction-of-Persian-Numbers/refs/heads/main/data_hoda.py
!wget -q https://github.com/MaryamMortazaviSefat/practice2_Prediction-of-Persian-Numbers/raw/refs/heads/main/dataset/Data_hoda_full.mat -P dataset

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, losses
from tensorflow.keras.models import Model
from data_hoda import load_hoda
import random

In [3]:
x_train,y_train,x_test,y_test=load_hoda(size=25)

In [4]:
x_train = x_train.astype('float32') / 255.
x_test = x_test.astype('float32') / 255.

In [26]:
x_train = x_train.reshape(-1, 25, 25, 1)
x_test = x_test.reshape(-1, 25, 25, 1)

In [28]:
x_test.shape

(200, 25, 25, 1)

In [29]:
class Cnn(Model):
    def __init__(self, num):
        super(Cnn, self).__init__()
        self.num=num


    def call(self, x):

        conv1 = layers.Conv2D(32, (3, 3), padding='same', activation='relu')(x)
        bn1 = layers.BatchNormalization()(conv1)
        pool1 = layers.MaxPooling2D(pool_size=(2, 2))(bn1)

        # Residual Block 1
        res1 = pool1
        conv2_1 = layers.Conv2D(32, (3, 3), padding='same')(pool1)
        bn2_1 = layers.BatchNormalization()(conv2_1)
        relu2_1 = tf.nn.relu(bn2_1)
        conv2_2 = layers.Conv2D(32, (3, 3), padding='same')(relu2_1)
        bn2_2 = layers.BatchNormalization()(conv2_2)
        residual1 = tf.keras.layers.add([bn2_2, res1])  # اتصالات جانبی
        relu_res1 = tf.nn.relu(residual1)

        pool2 = layers.MaxPooling2D(pool_size=(2, 2))(relu_res1)

        # Residual Block 2
        res2 = layers.Conv2D(64,(3,3),padding='same')(pool2)
        conv3_1 = layers.Conv2D(64, (3, 3), padding='same')(pool2)
        bn3_1 = layers.BatchNormalization()(conv3_1)
        relu3_1 = tf.nn.relu(bn3_1)
        conv3_2 = layers.Conv2D(64, (3, 3), padding='same')(relu3_1)
        bn3_2 = layers.BatchNormalization()(conv3_2)
        residual2 = tf.keras.layers.add([bn3_2, res2])  # اتصالات جانبی
        relu_res2 = tf.nn.relu(residual2)

        pool3 = layers.MaxPooling2D(pool_size=(2, 2))(relu_res2)
        flatten = layers.Flatten()(pool3)
        output = layers.Dense(self.num, activation='softmax')(flatten)

        return output


In [30]:
num_classes = 10
batch_size = 32
epochs = 10

model = Cnn(num=10)
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])


In [31]:
model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=epochs, batch_size=batch_size)
test_loss, test_acc = model.evaluate(x_test, y_test, verbose=2)

test_acc

Epoch 1/10


/usr/local/lib/python3.10/dist-packages/keras/src/backend/tensorflow/trainer.py:75: UserWarning: The model does not have any trainable weights.
  warnings.warn("The model does not have any trainable weights.")


ValueError: Exception encountered when calling Cnn.call().

[1mtf.function only supports singleton tf.Variables created on the first call. Make sure the tf.Variable is only created once or created outside tf.function. See https://www.tensorflow.org/guide/function#creating_tfvariables for more information.[0m

Arguments received by Cnn.call():
  • x=tf.Tensor(shape=(None, 25, 25, 1), dtype=float32)

In [32]:
#keras.utils.plot_model(autoencoder.encoder,show_shapes=True)

In [33]:
#keras.utils.plot_model(autoencoder.decoder,show_shapes=True)